In [92]:
acids = [
    ['AlaA', ['GCT', 'GCC', 'GCA', 'GCG']],
['ArgR',  ['CGT', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG']],
['AsnN',  ['AAT', 'AAC']],
['AspD',  ['GAT', 'GAC']],
['CysC',  ['TGT', 'TGC']],
['GlnQ',  ['CAA', 'CAG']],
['GluE',  ['GAA', 'GAG']],
['GlyG',  ['GGT', 'GGC', 'GGA', 'GGG']],
['HisH',  ['CAT', 'CAC']],   
['IleI',  ['ATT', 'ATC', 'ATA']],       
['LeuL',  ['TTA', 'TTG', 'CTT', 'CTC', 'CTA', 'CTG']],
['LysK',  ['AAA', 'AAG']],
['MetM',  ['ATG']],
['PheF',  ['TTT', 'TTC']],
['ProP',  ['CCT', 'CCC', 'CCA', 'CCG']],
['SerS',  ['TCT', 'TCC', 'TCA', 'TCG', 'AGT', 'AGC']],
['ThrT',  ['ACT', 'ACC', 'ACA', 'ACG']],
['TrpW',  ['TGG']],
['TyrY',  ['TAT', 'TAC']],
['ValV',  ['GTT', 'GTC', 'GTA', 'GTG']],
['STOP',  ['TAA', 'TGA', 'TAG']]
    ]

#Note that start codon has been expelled - for simplicity, it also codes for MetM
#LeuL to ValV - replace first T with G, coincidence?

In [93]:
# IN: Codon, OUT: Acid name
def codonToAcid(codon):
    data = [acids[x][1] for x in range(0,len(acids))]
    
    for sublist in data:
        if codon in sublist:
            return acids[data.index(sublist)][0]
            break

In [137]:
bases = ['A', 'C', 'T', 'G']
codonslist = [acids[x][1] for x in range(0,len(acids))]
codons = [item for sublist in codonslist for item in sublist]
acidssep = [acids[i][0] for i in range(0,len(acids))]

In [95]:
# Levenshtein distance one
def areNeighbors(a, b):
    count = 0
    for c1,c2 in zip(a,b):
        if c1 == c2:
            count += 1
    if count == 2:
        return True
    return False
            

In [96]:
adjMatrix = []
for cod1 in codons:
    newrow = []
    for cod2 in codons:
        newrow.append(1 if areNeighbors(cod1, cod2) else 0)
    adjMatrix.append(newrow)

In [126]:
import numpy as np
np.set_printoptions(threshold=np.inf)
adjMatrix = np.asarray(adjMatrix)
transMatrix = 1/9 * adjMatrix
adjMatrix

array([[0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,

In [171]:
# IN: Two acids OUT: Number of transitions between them
def prechod(acid1, acid2):
    # Get all rows of adjacency matrix for given acid
    pom = [adjMatrix[n].tolist() for n in [codons.index(i) for i in acids[acidssep.index(acid1)][1]]]
    # Sum the rows to get frequencies
    frequencies = [sum(i) for i in zip(*pom)]
    # Sum those leading to acid2
    ## First get the entries to sum
    entries = [codons.index(i) for i in acids[acidssep.index(acid2)][1]]
    count = 0
    for i in entries:
        count += frequencies[i]
    return count


In [212]:
# Frequency
freqMatrix = []
for acid1 in acidssep:
    newRow = []
    for acid2 in acidssep:
        newRow.append(prechod(acid1,acid2))
    freqMatrix.append(newRow)
    
freqMatrix

[[12, 0, 0, 2, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 4, 4, 4, 0, 0, 4, 0],
 [0, 18, 0, 0, 2, 2, 0, 6, 2, 1, 4, 2, 1, 0, 4, 6, 2, 2, 0, 0, 2],
 [0, 0, 2, 2, 0, 0, 0, 0, 2, 2, 0, 4, 0, 0, 0, 2, 2, 0, 2, 0, 0],
 [2, 0, 2, 2, 0, 0, 4, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0],
 [0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 4, 0, 2, 2, 0, 2],
 [0, 2, 0, 0, 0, 2, 2, 0, 4, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 2],
 [2, 0, 0, 4, 0, 2, 2, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2],
 [4, 6, 0, 2, 2, 0, 2, 12, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 4, 1],
 [0, 2, 2, 2, 0, 4, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0],
 [0, 1, 2, 0, 0, 0, 0, 0, 0, 6, 4, 1, 3, 2, 0, 2, 3, 0, 0, 3, 0],
 [0, 4, 0, 0, 0, 2, 0, 0, 2, 4, 18, 0, 2, 6, 4, 2, 0, 1, 0, 6, 3],
 [0, 2, 4, 0, 0, 2, 2, 0, 0, 1, 0, 2, 1, 0, 0, 0, 2, 0, 0, 0, 2],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 3, 2, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0],
 [0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 6, 0, 0, 2, 0, 2, 0, 0, 2, 2, 0],
 [4, 4, 0, 0, 0, 2, 0, 0, 2, 0, 4, 0, 0, 0, 12, 4, 4, 0, 0, 0, 0],
 [4, 

In [198]:
# Number of self-transitions divided by number of acids
np.asarray([freqMatrix[i][i] for i in range(0,len(freqMatrix))]) / np.asarray(list(map(len,[acids[x][1] for x in range(0,len(acids))])))

array([ 3.        ,  3.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  3.        ,  1.        ,  2.        ,
        3.        ,  1.        ,  0.        ,  1.        ,  3.        ,
        2.33333333,  3.        ,  0.        ,  1.        ,  3.        ,
        1.33333333])

In [222]:
# Create a transition matrix
freqMatrix = np.asarray(freqMatrix)
transMatrix = np.asmatrix([freqMatrix[i]/sum(freqMatrix[i]) for i in range(0,len(freqMatrix))])

In [242]:
# Getting to acids from the worst ones
initial = np.array([0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0])
print(initial * transMatrix**1)
print(initial * transMatrix **2)
print(initial * transMatrix **3)

[[ 0.          0.11111111  0.          0.          0.          0.          0.
   0.          0.          0.33333333  0.22222222  0.11111111  0.          0.
   0.          0.          0.11111111  0.          0.          0.11111111
   0.        ]]
[[ 0.02469136  0.08436214  0.05555556  0.00617284  0.00411523  0.02469136
   0.01851852  0.02469136  0.01234568  0.11728395  0.15020576  0.03497942
   0.05967078  0.05555556  0.03703704  0.06378601  0.09053498  0.00823045
   0.          0.09876543  0.02880658]]
[[ 0.04359092  0.0829904   0.03208352  0.02057613  0.01981405  0.02431032
   0.02042372  0.03886603  0.02446273  0.09106843  0.13664076  0.04229538
   0.02735863  0.03985673  0.05136412  0.08085658  0.07696997  0.01036427
   0.02149063  0.08367627  0.03094041]]


In [244]:
initial = np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0])
print(initial * transMatrix**1)
print(initial * transMatrix **2)

[[ 0.          0.22222222  0.          0.          0.22222222  0.          0.
   0.11111111  0.          0.          0.11111111  0.          0.          0.
   0.          0.11111111  0.          0.          0.          0.
   0.22222222]]
[[ 0.02057613  0.15432099  0.00411523  0.00617284  0.06378601  0.02880658
   0.02263374  0.09876543  0.01234568  0.01646091  0.08230453  0.02469136
   0.00823045  0.04115226  0.03292181  0.13786008  0.02057613  0.05658436
   0.0617284   0.02469136  0.08127572]]


In [249]:
for i in range(0, len(acids)):
    initial = np.zeros(21)
    initial[i] = 1
    print(initial * transMatrix **2)

[[ 0.18106996  0.04938272  0.01646091  0.04938272  0.01440329  0.01234568
   0.04938272  0.10288066  0.01234568  0.02263374  0.03497942  0.01234568
   0.00617284  0.01028807  0.09465021  0.10288066  0.09876543  0.00514403
   0.01028807  0.09876543  0.0154321 ]]
[[ 0.03292181  0.18792867  0.01989026  0.01028807  0.04183813  0.03840878
   0.01714678  0.0877915   0.03155007  0.03120713  0.08504801  0.02812071
   0.01406036  0.01783265  0.07544582  0.10699588  0.05349794  0.02572016
   0.01783265  0.02469136  0.05178326]]
[[ 0.03292181  0.05967078  0.11728395  0.04938272  0.02057613  0.04938272
   0.04938272  0.01646091  0.04938272  0.0627572   0.03292181  0.05967078
   0.02777778  0.02469136  0.03292181  0.08024691  0.09876543  0.00205761
   0.05349794  0.02469136  0.05555556]]
[[ 0.09876543  0.0308642   0.04938272  0.11728395  0.01851852  0.04938272
   0.06790123  0.09876543  0.04938272  0.02160494  0.0308642   0.04938272
   0.00308642  0.01851852  0.02469136  0.04320988  0.02469136  0.0

In [287]:
# Stationary distribution
initial = np.zeros(21)
initial[0] = 1
stationary = initial * transMatrix**100000

In [296]:
[[acidssep[i], round(stationary[i],5)] for i in range(0, len(acidssep))]

[['AlaA', 0.0625],
 ['ArgR', 0.09375],
 ['AsnN', 0.03125],
 ['AspD', 0.03125],
 ['CysC', 0.03125],
 ['GlnQ', 0.03125],
 ['GluE', 0.03125],
 ['GlyG', 0.0625],
 ['HisH', 0.03125],
 ['IleI', 0.04687],
 ['LeuL', 0.09375],
 ['LysK', 0.03125],
 ['MetM', 0.01562],
 ['PheF', 0.03125],
 ['ProP', 0.0625],
 ['SerS', 0.09375],
 ['ThrT', 0.0625],
 ['TrpW', 0.01562],
 ['TyrY', 0.03125],
 ['ValV', 0.0625],
 ['STOP', 0.04687]]

In [293]:
stationary = stationary.tolist()[0]